# Library

In [1]:
import requests
from bs4 import BeautifulSoup as bs

import time, os, random
import pandas as pd
import re
import numpy as np

from selenium import webdriver
import chromedriver_autoinstaller as ca

from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

## chrome driver 설치

In [2]:
# USB error 메세지 발생 해결을 위한 코드
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [3]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'131'

In [4]:
# # 크롬 드라이버 확인 및 설치(처음 한번만 실행)
# ca.install(True)

## chrome driver 실행

In [5]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

## 페이지 접속

In [6]:
# 페이지 주소
url = 'https://shopping.naver.com/ns/home'

### requests 테스트
- 페이지 접속 가능 여부확인
    - 가능할 경우 출력 : <Response [200]>

In [7]:
# req = requests.get(url)
# print(req)
# html = req.content.decode('utf-8') # 한글 깨짐 해결 코드
# soup = bs(html, 'html.parser')
# soup.title.text

### selenium 작동

In [ ]:
driver.get(url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
# driver.maximize_window() # 브라우져 창 최대화

### 검색 창에 검색어 입력

In [9]:
search_keyword = '아이라인'
search_window_xpath = '//*[@id="gnb-gnb"]/div[2]/div/div[2]/div[1]/form/div/div[1]/div/div[1]/input' # 검색 입력창

search_window = driver.find_element(By.XPATH, search_window_xpath)
search_window.send_keys(search_keyword)
time.sleep(random.randrange(1, 3))
search_window.send_keys(Keys.ENTER)

In [11]:
# 스크롤해야 정보가 로드되는 페이지

# 페이지 스크롤 다운

last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 문서 높이 만큼 스크롤
    # driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") # # 문서 높이 만큼 스크롤(위 코드와 동일한 기능)
    time.sleep(random.uniform(1, 3))
    driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료

    new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_page_height == last_page_height:
        break
    else:
        last_page_height = new_page_height

## 🔧 입력 키워드 정보 수집
- 페이지 로드되어도 보고있는 화면 근처 정보만 수집되는걸로 보임
    1. 페이지 스크롤 -> 수집 -> 페이지 스크롤 반복
    2. 수집 후 중복 제거

In [28]:
page = driver.page_source
soup = bs(page, 'html.parser')
soup.title.text

'아이라인 : 네이버+ 스토어 검색'

In [29]:
# 상품 섹션
products_section = soup.select_one('div#composite-card-list')
len(products_section)

2

In [30]:
# 상품 리스트
product_list = products_section.select('li.compositeCardContainer_composite_card_container__jr8cb.composite_card_container div.basicProductCard_basic_product_card__TdrHT.basicProductCard_view_type_grid2__vKr1n')
len(product_list)

49

### 상세 페이지 링크

In [87]:
# 상품 상세 페이지
order_number = 0
product_list[order_number].select_one('a.basicProductCard_link__urzND._nlog_click._nlog_impression_element')['href']

'https://adcr.naver.com/adcr?x=ShI0EK/hmFfaH5W/CPwr8P///w==kVS8JpRJODxTgJSvHauwbacQb8jKmUBd61bMbkKvzwu3ORurFBOTBhCwaPCGNROFVMPNPj6qt9v8v7hlgg8yayaCUfVMMWou6wTx5LlReIDViD1deV8wV1kQYe7sWn/QXfCSKq7ApoXe8A8aSqjGtVdAV3K5Y3h31DPxCCXsMERpY8/VC0+pRYjWe0ziK1+L0Khxd7pUC+gkM+4m2TYGtU/Gno3jNzcw1iTFPBs8JPwZHthYr1yLC6cplHZOmFCFvf4hNYod3BWht2De80L6JqQXTCG7ip+M8h20BzmOr28A/cfsUFtn9L16ryY1XPBGRcxEar92r5pHx0wiIL7fMOhvSZUfujm0BTtGhvlH1AvUWA7aVHgBp1d2ecZoEooD1byTgT9/V6AExcMxGzd/pLPwII+6Ejmnd18GkNb6vCtqcmX5y/GMgFoZaw/kwyfG6zfcFAWfAUa/t9/JQUr6wzrQdpwvFAFo55IsZ5fySlyFs4S9nmQsLLaQeK27AjSu3/6kyeZ1T1uX3fZPSLUkel7vtb4RFdP3KWKB9tlpRRSrvXPssBH/JWOMGtS56xzZbNvGN4XEuL5S0fVz/MTVkuxUIepz9qOJfm5njLPRWY6HxpA/AhDGoA010mRXTzNCQ4eu7BY4jYyhQeC/Brgv3QeGLkQWK4Cs4721rF863xp4VlMH6yAZyROjCexQ7CPZHhaUxw9uIjm+9mZeDCrgQYsQvQiiZXetDXAkvQESEbsWvcR/Y8vpZtT4Vos4bahzI/bU/m17mGGbnZtQxOPFh6mGbNlFxQohYsb9dCx7jVnK7nuZgg7KhAClh/KO1Z4UNIHWFnHG77/YELwbfEQFElsBt0LtoL3P7MR80YX9F55YWQrKi+v0hz5nKC+lnH/hB#channelProductNo=7276573041'

### 🔧 이미지 링크
- 이미지 저장 기능 추가

In [34]:
# 상품 썸네일 이미지 링크
order_number = 0
product_list[order_number].select_one('div.productCardThumbnail_thumbnail__KzO1N img.autoFitImg_auto_fit_img__fIpj4.autoFitImg_full_height__QCTGq.productCardThumbnail_image__Li6iz.scale')['src']

'https://shopping-phinf.pstatic.net/main_8482107/84821073363.5.jpg?type=f300'

### 상품 정보 섹션

In [88]:
# 상품 정보 섹션
order_number = 0
product_info_section = product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc')
product_info_section

<div class="basicProductCardInformation_basic_product_card_information__7v_uc basicProductCardInformation_view_type_grid2__mlh6E" id="basic_product_card_information_7276573041"><div class="basicProductCardInformation_mall_area__xcGLH"><div class="basicProductCardInformation_advertisement_area__HzaQ_"><span class="advertisementTag_advertisement_tag__yIvim"><a class="advertisementTag_link__NAi19 _nlog_click" data-shp-area="prod.adinfo" data-shp-area-id="adinfo" data-shp-area-type="action" data-shp-inventory="prod" data-shp-nsc="svc.shopping.plusall" data-shp-page-key="100407255" data-shp-sti="shopping" href="#" role="button" target="_blank"><svg class="advertisementTag_icon__DdazQ advertisementTag_information__jKJtn" fill="none" height="18" viewbox="0 0 40 18" width="40" xmlns="http://www.w3.org/2000/svg"><rect height="17" rx="8.5" stroke="#D9D9DC" width="39" x="0.5" y="0.5"></rect><path d="M13.096 11.988c0 1.133-1.265 1.826-3.333 1.826s-3.311-.693-3.311-1.826 1.254-1.837 3.311-1.837 3.3

#### 상품 제목

In [40]:
# 상품 제목
print(product_info_section.select_one('strong.basicProductCardInformation_title__Bc_Ng').text)

나어릴때 워터프루프 브러쉬 아이라인


#### 🔧 상품 가격
- 숫자만 추출

In [41]:
# 상품 가격

print(product_info_section.select_one('span.priceTag_original_price__jyZRY').text)
print(product_info_section.select_one('span.priceTag_discount__F_ZXz').text)
print(product_info_section.select_one('span.priceTag_inner_price__TctbK').text)

할인 전 판매가14,000원
39%할인
8,500원


#### 🔧 배송비
- 숫자만 추출

In [42]:
product_info_section.select_one('div.productCardPrice_delivery_price__AiyD2').text

'배송비 3,000원'

#### 종합

In [54]:
order_number = 0

# 상세 페이지 링크
print(product_list[order_number].select_one('a.basicProductCard_link__urzND._nlog_click._nlog_impression_element')['href'])


# 이미지 링크
print(product_list[order_number].select_one('div.productCardThumbnail_thumbnail__KzO1N img.autoFitImg_auto_fit_img__fIpj4.autoFitImg_full_height__QCTGq.productCardThumbnail_image__Li6iz.scale')['src'])

# 상품 정보 섹션
product_info_section = product_list[order_number].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc')

# 상품 제목
print(product_info_section.select_one('strong.basicProductCardInformation_title__Bc_Ng').text)

# 원래 상품 가격
try:
    print(product_info_section.select_one('span.priceTag_original_price__jyZRY').text)
except:
    print('할인없음')
# 할인률
try:
    print(product_info_section.select_one('span.priceTag_discount__F_ZXz').text)
except:
    print('할인률없음')
# 현재 가격
print(product_info_section.select_one('span.priceTag_inner_price__TctbK').text)
# 배송비비
try:
    print(product_info_section.select_one('div.productCardPrice_delivery_price__AiyD2').text)
except:
    print('배송비없음')

https://adcr.naver.com/adcr?x=ShI0EK/hmFfaH5W/CPwr8P///w==kVS8JpRJODxTgJSvHauwbacQb8jKmUBd61bMbkKvzwu3ORurFBOTBhCwaPCGNROFVMPNPj6qt9v8v7hlgg8yayaCUfVMMWou6wTx5LlReIDViD1deV8wV1kQYe7sWn/QXfCSKq7ApoXe8A8aSqjGtVdAV3K5Y3h31DPxCCXsMERpY8/VC0+pRYjWe0ziK1+L0Khxd7pUC+gkM+4m2TYGtU/Gno3jNzcw1iTFPBs8JPwZHthYr1yLC6cplHZOmFCFvf4hNYod3BWht2De80L6JqQXTCG7ip+M8h20BzmOr28A/cfsUFtn9L16ryY1XPBGRcxEar92r5pHx0wiIL7fMOhvSZUfujm0BTtGhvlH1AvUWA7aVHgBp1d2ecZoEooD1byTgT9/V6AExcMxGzd/pLPwII+6Ejmnd18GkNb6vCtqcmX5y/GMgFoZaw/kwyfG6zfcFAWfAUa/t9/JQUr6wzrQdpwvFAFo55IsZ5fySlyFs4S9nmQsLLaQeK27AjSu3/6kyeZ1T1uX3fZPSLUkel7vtb4RFdP3KWKB9tlpRRSrvXPssBH/JWOMGtS56xzZbNvGN4XEuL5S0fVz/MTVkuxUIepz9qOJfm5njLPRWY6HxpA/AhDGoA010mRXTzNCQ4eu7BY4jYyhQeC/Brgv3QeGLkQWK4Cs4721rF863xp4VlMH6yAZyROjCexQ7CPZHhaUxw9uIjm+9mZeDCrgQYsQvQiiZXetDXAkvQESEbsWvcR/Y8vpZtT4Vos4bahzI/bU/m17mGGbnZtQxOPFh6mGbNlFxQohYsb9dCx7jVnK7nuZgg7KhAClh/KO1Z4UNIHWFnHG77/YELwbfEQFElsBt0LtoL3P7MR80YX9F55YWQrKi+v0hz5nKC+lnH/hB#channelProductNo=7276573041
https://shopping

# 🔧 수집한 데이터 전처리
- 원래가격, 할인률, 현재가격 숫자만 표기하도록 정제
- 원래가격, 할인률 nan 값 처리된 부분 확인

In [59]:
num_list = []
detail_link_list = []
img_src_list = []
title_list = []
original_price_list = []
discount_rate_list = []
price_list = []
delivery_price_list = []

for i in range(len(product_list)):
    num = i + 1

    # 상세 페이지 링크
    try:
        detail_link = product_list[i].select_one('a.basicProductCard_link__urzND._nlog_click._nlog_impression_element')['href']
    except:
        print(i)
        # print('상세페이지링크없음')
    # 이미지 링크
    try:
        img_src = product_list[i].select_one('div.productCardThumbnail_thumbnail__KzO1N img.autoFitImg_auto_fit_img__fIpj4.autoFitImg_full_height__QCTGq.productCardThumbnail_image__Li6iz.scale')['src']
    except:
        print(i)
        # print('이미지링크없음')
    
    # 상품 정보 섹션
    product_info_section = product_list[i].select_one('div.basicProductCardInformation_basic_product_card_information__7v_uc')
    
    ## 상품 제목
    try:
        title = product_info_section.select_one('strong.basicProductCardInformation_title__Bc_Ng').text
    except:
        print(i)
        # print('상품제목없음')
    
    ## 원래가격
    try:
        original_price = product_info_section.select_one('span.priceTag_original_price__jyZRY').text
    except:
        original_price = np.nan
        # print('원래가격없음')
    ## 할인률
    try:
        discount_rate = product_info_section.select_one('span.priceTag_discount__F_ZXz').text
    except:
        discount_rate = np.nan
        # print('할인률없음')
    ## 현재가격
    try:
        price = product_info_section.select_one('span.priceTag_inner_price__TctbK').text
    except:
        print(i)
        # print('현재가격없음')
    ## 배송비
    try:
        delivery_price = product_info_section.select_one('div.productCardPrice_delivery_price__AiyD2').text
    except:
        delivery_price = np.nan
        # print('배송비없음')

    # 정보 리스트에 담기
    num_list.append(num)
    detail_link_list.append(detail_link)
    img_src_list.append(img_src)
    title_list.append(title)
    original_price_list.append(original_price)
    discount_rate_list.append(discount_rate)
    delivery_price_list.append(delivery_price)
    price_list.append(price)


In [60]:
len(num_list), len(detail_link_list), len(img_src_list), len(title_list), len(original_price_list), len(discount_rate_list), len(price_list), len(delivery_price_list)

(49, 49, 49, 49, 49, 49, 49, 49)

In [62]:
data_dict = {
    '번호':num_list, 
    '상세페이지링크':detail_link_list,
    '이미지링크':img_src_list,
    '제품제목':title_list,
    '원래가격':original_price_list,
    '할인률':discount_rate_list,
    '현재가격':price_list,
    '배송비':delivery_price_list
    }
df = pd.DataFrame(data_dict)
df.head()

,번호,상세페이지링크,이미지링크,제품제목,원래가격,할인률,현재가격,배송비
0,1,https://adcr.naver.com/adcr?x=ShI0EK/hmFfaH5W/...,https://shopping-phinf.pstatic.net/main_848210...,나어릴때 워터프루프 브러쉬 아이라인,"할인 전 판매가14,000원",39%할인,"8,500원","배송비 3,000원"
1,2,https://adcr.naver.com/adcr?x=/+J95PYl7YiFUUXv...,https://searchad-phinf.pstatic.net/MjAyMjExMjF...,1+1 미크릿 워셔블 비건 슈퍼픽서 아이라이너,"할인 전 판매가24,000원",31%할인,"16,500원",NaN
2,3,https://adcr.naver.com/adcr?x=k24YgNHEghYRsyK7...,https://searchad-phinf.pstatic.net/MjAyMzA1MDh...,메리쏘드 릴타투 브러쉬 펜라이너 붓펜 아이라인,"할인 전 판매가11,000원",5%할인,"10,400원","배송비 3,000원"
3,4,https://adcr.naver.com/adcr?x=6CXXG4DcFj/K1vNG...,https://shopping-phinf.pstatic.net/main_825922...,[공식]메이블린 라인 타투 아이 라이너 크레용,"할인 전 판매가13,000원",25%할인,"9,750원",NaN
4,5,https://smartstore.naver.com/main/products/551...,https://shopping-phinf.pstatic.net/main_830583...,번지지않는 붓펜 아이라이너 점막 안번지는 저자극 그리기쉬운 아이라인 펜슬 붓 트임,"할인 전 판매가12,900원",23%할인,"9,900원","배송비 3,000원"


In [63]:
df.tail()

,번호,상세페이지링크,이미지링크,제품제목,원래가격,할인률,현재가격,배송비
44,45,https://smartstore.naver.com/main/products/886...,https://shopping-phinf.pstatic.net/main_864141...,클리오 수퍼 프루프 펜라이너 0.55ml 아이라이너 아이라인 펜슬,NaN,NaN,"12,780원","배송비 3,500원"
45,46,https://smartstore.naver.com/main/products/865...,https://shopping-phinf.pstatic.net/main_862023...,아이라인그리기 눈화장가이드 애교살만들기 마스카라가드,"할인 전 판매가8,000원",60%할인,"3,200원","배송비 3,000원"
46,47,https://smartstore.naver.com/main/products/831...,https://shopping-phinf.pstatic.net/main_858610...,발색력좋은 펜슬 아이라인 번짐없는 토니모리 아이라인 롱브러쉬 브라운,"할인 전 판매가27,900원",46%할인,"14,900원","배송비 2,500원"
47,48,https://smartstore.naver.com/main/products/817...,https://shopping-phinf.pstatic.net/main_857237...,애교살 아이라인 가이드 실리콘 라이너 그리기 도구,"할인 전 판매가2,900원",10%할인,"2,610원","배송비 3,000원"
48,49,https://smartstore.naver.com/main/products/513...,https://shopping-phinf.pstatic.net/main_826788...,워터프루프 오앤에이 아이라이너 롱라스팅 브러쉬 초슬림 아이라인 펜슬 3색상,"할인 전 판매가16,000원",65%할인,"5,500원","배송비 3,000원"


In [67]:
df.loc[df['현재가격'].isnull()]

,번호,상세페이지링크,이미지링크,제품제목,원래가격,할인률,현재가격,배송비


In [69]:
df.loc[df['배송비'].isnull()]

,번호,상세페이지링크,이미지링크,제품제목,원래가격,할인률,현재가격,배송비
1,2,https://adcr.naver.com/adcr?x=/+J95PYl7YiFUUXv...,https://searchad-phinf.pstatic.net/MjAyMjExMjF...,1+1 미크릿 워셔블 비건 슈퍼픽서 아이라이너,"할인 전 판매가24,000원",31%할인,"16,500원",NaN
3,4,https://adcr.naver.com/adcr?x=6CXXG4DcFj/K1vNG...,https://shopping-phinf.pstatic.net/main_825922...,[공식]메이블린 라인 타투 아이 라이너 크레용,"할인 전 판매가13,000원",25%할인,"9,750원",NaN
5,6,https://smartstore.naver.com/main/products/906...,https://shopping-phinf.pstatic.net/main_866077...,잘안지워지는 점막 지워지지않는 수영장 음영 얇은 아이라인 아이라이너 펜슬,"할인 전 판매가17,800원",11%할인,"15,800원",NaN
6,7,https://smartstore.naver.com/main/products/399...,https://shopping-phinf.pstatic.net/main_978286...,1+1 리르 룩스 젤 펜슬 아이라이너 아이라인 아이메이크업 0.5g,"할인 전 판매가20,000원",50%할인,"9,900원",NaN
7,8,https://smartstore.naver.com/main/products/467...,https://shopping-phinf.pstatic.net/main_822170...,아임유니 울트라 슬림 워터프루프 젤 아이라이너 펜슬 8컬러 안번지는 점막 아이라인 컬러,"할인 전 판매가11,000원",46%할인,"5,900원",NaN
10,11,https://smartstore.naver.com/main/products/504...,https://shopping-phinf.pstatic.net/main_825922...,메이블린 뉴욕 라인 타투 아이 라이너 크레용 0.4g,"할인 전 판매가13,000원",25%할인,"9,750원",NaN
12,13,https://smartstore.naver.com/main/products/562...,https://shopping-phinf.pstatic.net/main_831715...,블레스문 블랙홀 라스팅 워터프루프 아이라이너 (3Colors) 라인괴물 지속력 끝판...,NaN,NaN,"24,000원",NaN
28,29,https://smartstore.naver.com/main/products/502...,https://shopping-phinf.pstatic.net/main_825677...,1+1미크릿 비건 안번지는 미온수클렌징 아이라이너 0.6g(다크브라운/블랙),"할인 전 판매가24,000원",31%할인,"16,500원",NaN
39,40,https://smartstore.naver.com/main/products/733...,https://shopping-phinf.pstatic.net/main_848769...,헤라 이지 스타일링 아이라이너 0.5g 붓펜 아이라인 워터프루프,NaN,NaN,"21,780원",NaN
40,41,https://smartstore.naver.com/main/products/455...,https://shopping-phinf.pstatic.net/main_821019...,(3개)비러브 마이라인 펜 아이라이너 블랙,"할인 전 판매가11,400원",9%할인,"10,370원",NaN


In [73]:
sum(df['배송비'].isnull())

10

# 데이터 출력

In [76]:
current_date = time.strftime('%Y%m%d')
current_date

'20241205'

In [77]:
current_datetime = time.strftime('%Y%m%d_%p_%I%M%S')
current_datetime

'20241205_PM_053801'

In [78]:
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver'

In [81]:
# 수집한 파일 저장할 폴더 생성
crawled_folder_path = os.path.join(code_path, 'crawled_data', current_date)
crawled_folder_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver\\crawled_data\\20241205'

In [82]:
os.makedirs(crawled_folder_path, exist_ok=True)

In [85]:
# 엑셀 파일로 출력
df.to_excel(os.path.join(crawled_folder_path, f'naver_{search_keyword}_{current_datetime}.xlsx'), index=False)

# END